In [1]:
!pip install crewai-tools==0.25.0
!pip install crewai==0.86.0
!pip install langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.5/514.5 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 42.2

In [23]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# **Simple Blog**

In [31]:
from crewai import LLM

llm = LLM(
    api_key="Your Api Key",
    model="groq/llama3-70b-8192"
)

## Agents

In [32]:
from crewai import Agent, Task, Crew
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)
writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "Write the blog in a well-readable manner, partitioning it into sections as per the requirement. Ensure there are at least 5-6 sections, making the entire blog lengthier.",
    allow_delegation=False,
    verbose=True
)
editor = Agent(
    llm=llm,
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Tasks

In [33]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Define Crew

In [34]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

Kick off

In [35]:
result = crew.kickoff(inputs={"topic": "Write a blog about us politcs"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Write a blog about us politcs.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan: "Write a Blog about US Politics"**

**I. Audience Analysis**

Target Audience:

* Demographics: Americans aged 25-55, interested in current events and politics
* Interests: Staying informed about political news, analyzing policies, and making informed decisions
* Pain Points: Navigating biased news sources, staying up-to-date with latest developments, and understanding complex political issues
* Goals: To gain a deeper understanding of US politics, stay informed about current events, and engage in meaningful discussions

**II. Content Outline**

**Introduction**

* Ho

Display

In [36]:
from IPython.display import Markdown
Markdown(result.raw)

**The Evolving Landscape of US Politics: Trends, Players, and News**
=================================================================================

**Staying Informed in a Rapidly Changing Political Climate**
--------------------------------------------------------

In today's fast-paced political environment, it's more important than ever to stay informed about the latest trends, key players, and noteworthy news in US politics. With the 24-hour news cycle and social media, it's easy to get caught up in the noise and miss the bigger picture. However, by taking a step back and examining the current state of US politics, we can gain a deeper understanding of the complex issues shaping our country's future.

In this blog, we'll explore the latest trends, key players, and noteworthy news in US politics, empowering you to make informed decisions and engage in meaningful discussions. By staying up-to-date on the latest developments, we can navigate the complex landscape of US politics and become more informed citizens.

### **Section 1: Latest Trends in US Politics**

#### **The Rise of Progressive Politics**

The progressive movement has experienced significant growth in recent years, with many Americans embracing progressive ideals and policies. According to a 2020 Pew Research Center study, "The Rise of the Progressive Left," 57% of Democrats identify as liberal or progressive, up from 43% in 2016. This shift is reflected in the election of progressive politicians such as Alexandria Ocasio-Cortez and Bernie Sanders, who have become prominent voices in the Democratic Party.

A 2022 CNN article, "Progressive Democrats see surge in fundraising, voter engagement," noted that progressive Democrats have seen a significant increase in fundraising and voter engagement, suggesting that the movement is gaining momentum. This trend is likely to continue shaping the political landscape in the coming years.

#### **The Evolution of Conservative Politics**

Conservative politics have also undergone significant changes in recent years. A 2020 Brookings Institution study, "The Future of the Republican Party," highlighted the shift towards a more populist and nationalist conservatism, exemplified by the presidency of Donald Trump. This shift has led to a re-evaluation of conservative policies and priorities, with some Republicans embracing more moderate stances.

A 2022 NPR article, "Conservative Politics in the Age of Trump," noted that conservative politics have become more divisive, with some Republicans embracing Trump's populist ideology, while others have distanced themselves from it. As the Republican Party continues to evolve, it will be important to monitor these trends and their impact on US politics.

### **Section 2: Key Players in US Politics**

#### **Influential Politicians**

From the presidency to Congress, there are several influential politicians shaping the political landscape in the United States. Joe Biden, Nancy Pelosi, and Mitch McConnell are just a few examples of prominent politicians who have had a significant impact on policy decisions and elections.

According to a 2022 Gallup poll, Joe Biden and Nancy Pelosi were among the most admired people in the world, reflecting their influence and popularity among Americans. A 2022 CNN article, "The Most Influential People in Politics," highlighted the significant role played by politicians in shaping public policy and opinion.

#### **Behind-the-Scenes Players**

While politicians often take center stage, there are many behind-the-scenes players who wield significant influence in US politics. Think tanks, lobbyists, and special interest groups play a crucial role in shaping policy decisions, often working behind the scenes to advance their agendas.

A 2022 Center for Responsive Politics study found that lobbying expenditures have increased significantly in recent years, with many industries and organizations seeking to influence policy decisions. A 2022 The Hill article, "The Most Powerful People in Washington," highlighted the significant influence of these behind-the-scenes players in shaping the political agenda.

### **Section 3: Noteworthy News and Updates**

#### **Recent Elections and Impacts**

The 2022 Midterm elections saw significant gains for Democrats, with many progressive candidates winning elections. According to a 2022 US Election Project study, the elections saw a significant increase in voter turnout, particularly among young people and minority voters.

A 2022 Pew Research Center study, "The Political Landscape Following the 2022 Midterms," found that the elections had significant implications for policy decisions, with Democrats gaining control of the House of Representatives. As we look to the future, it will be important to monitor the impact of these elections on US politics.

#### **Upcoming Political Events**

With the 2024 presidential election on the horizon, there are many significant political events to look out for. From primaries to debates to conventions, these events will shape the political landscape and influence policy decisions.

According to a 2022 Federal Election Commission study, the 2024 election cycle is expected to be one of the most expensive in history, with many candidates vying for attention and support. A 2022 New York Times article, "The 2024 Election: What to Expect," highlighted the significant implications of the election for the future of American politics.

### **Conclusion**

In conclusion, the landscape of US politics is constantly evolving, with new trends, players, and news emerging every day. By staying informed about the latest developments, we can gain a deeper understanding of the complex issues shaping our country's future. Remember to stay engaged, stay informed, and make your voice heard at the ballot box.

**Call to Action:** Stay informed about the latest trends, key players, and noteworthy news in US politics. Engage in meaningful discussions with others, and make informed decisions at the ballot box. Together, we can build a more informed and engaged citizenry.

# **Simple HTML webpage**

Define Custom Tools

In [37]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

Browse Google Image URLs

In [38]:
!pip install serpapi==0.1.5

TOOL 1: Browse Google Images

In [46]:
import serpapi
class GetImagesInput(BaseModel):
    """Image keyword to search for getting image urls from the web"""
    image_keyword: str = Field(..., description="Image keyword to search for getting image urls from the web")

class GetImagesFromWeb(BaseTool):
    name: str = "Get_relevant_image_urls"
    description: str = "Search for the relevant image urls on web based on the given keyword"
    args_schema: Type[BaseModel] = GetImagesInput

    def _run(self, image_keyword: str) -> list:
        print("GetImagesFromWeb tool called")
        urlList = []
        try:
          params = {
            "q": str(image_keyword),
            "engine": "google_images",
            "ijn": "0",
            "api_key": "Your Api Key"
          }
          search = serpapi.search(params)
          for eachObject in search['images_results'][:10]:
            urlList.append(eachObject['original'])
          print("urlList is")
          print(urlList)
          return urlList
        except Exception as e:
          return urlList
# Instantiate the tool
get_image_urls_tool = GetImagesFromWeb()

TOOL 2: Save HTML file tool

In [47]:
class SaveHtmlFileInput(BaseModel):
    """Input schema for saving an HTML file."""
    html_page_content: str = Field(..., description="HTML file contents to save.")

class SaveHtmlFileTool(BaseTool):
    name: str = "Save_html_file"
    description: str = "Saves the given HTML content to a file named 'mywebpage.html'."
    args_schema: Type[BaseModel] = SaveHtmlFileInput

    def _run(self, html_page_content: str) -> str:
        filename = "mywebpage.html"
        try:
          with open(filename, "w", encoding="utf-8") as file:
              file.write(html_page_content)
          return f"File saved successfully as {filename}"
        except Exception as e:
            return f"Error saving file: {str(e)}"

# Instantiate the tool
save_file_tool = SaveHtmlFileTool()

Web Developer Agent

In [48]:
web_developer = Agent(
    llm=llm,
    role = "Senior Web developer",
    goal = "Compile the results into a HTML file.",
    backstory = "You are a competent Web developer writing HTML and CSS with decades of experience. "
                "You receives a blog post from the editor "
                "Your Goal is the convert the edited final draft of the blog post in a well structured HTML file "
                "The web page should be desgined beutifully using js or css"
                "You must include images(use image urls from web) and background colores which are relevant to the topic and content mentioned in the input file."
                "Pass short relevant keywords like 'indian cricket','America','Machine learning'..etc to the given `Get_relevant_image_urls` tool fr getting relevant image urls from web."
                "Dont pass long keywords or sentences to the `Get_relevant_image_urls` tool"
                "The output will be a single HTML file all the images,fonts, css and js should be embedded inside that file"
                "The webpage must be colorfull and attractive. Add background color and font color based on the context of the blog. But dont keep it as default"
                "Save the HTML file in the folder with name current_webpage.html use given `Save_html_file` tool for saving.",
    tools = [save_file_tool,get_image_urls_tool],
    allow_delegation=False,
    verbose=True
)

Web Developer Task

In [49]:
webpage_creation = Task(
    description=("Convert the given edited final draft of the blog post into a html file"),
    expected_output="A well-desgined HTML page which contain both images and texts and looks very pretty"
                    "The output will be a single HTML file all the images,fonts, css and js should be embedded inside that file. Pass short relevant keywords to the given tool and use given tool for getting relevant image urls from web."
                    "The webpage must be colorfull and attractive. Add background color and font color based on the context of the blog. But dont keep it as default"
                    "Save the HTML file in the folder with name current_webpage.html use python tool for saving.",

    agent=web_developer
)

Redefine the Crew

In [50]:
crew = Crew(
    agents=[planner, writer, editor,web_developer],
    tasks=[plan, write, edit,webpage_creation],
    verbose=True
)

Kick Off

In [ ]:
result = crew.kickoff(inputs={"topic": "Malayam Cinema History"})

In [45]:
!pip freeze

absl-py==1.4.0
accelerate==1.8.1
aiofiles==24.1.0
aiohappyeyeballs==2.6.1
aiohttp==3.11.15
aiosignal==1.4.0
alabaster==1.0.0
albucore==0.0.24
albumentations==2.0.8
ale-py==0.11.1
alembic==1.16.3
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.9.0
appdirs==1.4.4
argon2-cffi==25.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.2
arviz==0.21.0
asgiref==3.9.1
astropy==7.1.0
astropy-iers-data==0.2025.7.7.0.39.39
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
auth0-python==4.10.0
autograd==1.8.0
babel==2.17.0
backcall==0.2.0
backoff==2.2.1
backports.tarfile==1.2.0
backrefs==5.9
bcrypt==4.3.0
beautifulsoup4==4.13.4
betterproto==2.0.0b6
bigframes==2.8.0
bigquery-magics==0.9.0
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blobfile==3.0.0
blosc2==3.5.1
bokeh==3.7.3
Bottleneck==1.4.2
bqplot==0.12.45
branca==0.8.1
build==1.2.2.post1
CacheControl==0.14.3
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.6.15
cffi==1.17.1
cfgv==3.4.0
chardet==5.2.0
charset-no